In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, IBMQ
from qiskit.tools.visualization import plot_histogram

In [3]:
import qiskit.ignis.mitigation as mit
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

In [4]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')
print(provider)
device = provider.backend.ibmq_brooklyn
backend = provider.get_backend('ibmq_brooklyn')

<AccountProvider for IBMQ(hub='ibm-q-utokyo', group='internal', project='hirashi-jst')>


In [5]:
# qubit layout
vertex_list =  [33, # 0
                32, # 1
                25, 31, # 2, 3
                34, 19, 39, # 4, 5, 6
                30, 35, 18, 45, # 7, 8, 9, 10
                20, 29, 40, 17, 46, # 11, 12, 13, 14, 15
                36, 44, 21, 28, 49, 16, 47, # 16, 17, 18, 19, 20, 21, 22
                24, 11, 37, 43, 12, 27, 50, 15, 53, # 23, 24, 25, 26, 27, 28, 29, 30, 31
                22, 48,  4, 26, 52,  8, 38, 51, 14, 60, # 32, 33, 34, 35, 36, 37, 38, 39, 40, 41
                42, 23,  3, 56,  7, 41, 54, 13, 59, # 42, 43, 44, 45, 46, 47, 48, 49, 50
                 5,  9, 61,  2, 55,  6, 64, 10, 58, # 51, 52, 53, 54, 55, 56, 57, 58, 59
                57, 62,  1, 63,  0] # 60, 61, 62, 63, 64
assert len(vertex_list) == len(set(vertex_list))
max_size = len(vertex_list) # largest graph size
shots_per_circuit = 1 << 13 # shots per one execution
print("max_size:", max_size)
print("shots_per_circuit:", shots_per_circuit)

max_size: 65
shots_per_circuit: 8192


In [6]:
# indicate how to connect qubits
order = [[0,1], # 2
         [0,2],  [1,3], # 3
         [0,4],  [2,5],  [3,6], # 4
         [3,7],  [4,8],  [5,9],  [6,10], # 5
         [5,11], [7,12], [8,13], [9,14], [10,15], # 6
         [8,16], [10,17],[11,18],[12,19],[13,20],[14,21],[15,22], # 7
         [12,23],[14,24],[16,25],[17,26],[18,27],[19,28],[20,29],[21,30],[22,31], # 8
         [18,32],[20,33],[24,34],[25,35],[26,36],[27,37],[28,38],[29,39],[30,40],[31,41], # 9
         [26,42],[32,43],[34,44],[36,45],[37,46],[38,47],[39,48],[40,49],[41,50], # 10
         [34,51],[37,52],[41,53],[44,54],[45,55],[46,56],[48,57],[49,58],[50,59], # 11
         [45,60],[53,61],[54,62],[57,63],[58,64]] # 12
print(len(order))

64


In [7]:
qubit_depths = [0, 
                1, 
                2, 2,
                3, 3, 3, 
                4, 4, 4, 4, 
                5, 5, 5, 5, 5, 
                6, 6, 6, 6, 6, 6, 6, 
                7, 7, 7, 7, 7, 7, 7, 7, 7,
                8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
                9, 9, 9, 9, 9, 9, 9, 9, 9,
                10,10,10,10,10,10,10,10,10,
                11,11,11,11,11]
print(len(qubit_depths))

65


# ghz circuits

In [8]:
def ghz_state(size, order, qubit_depths):
    qc = QuantumCircuit(size)
    qc.h(0)
    if size <= 1:
        return qc
    depth = 0
    for i, j in order:
        if depth != qubit_depths[j]:
            # qc.barrier()
            depth += 1
        qc.cx(i, j)
    return qc

In [9]:
def ghz_state_with_measurement(size, order, qubit_depths):
    qc = QuantumCircuit(size)
    qc.compose(ghz_state(size, order, qubit_depths), qubits = range(size), inplace = True)
    qc.measure_all()
    return qc

In [10]:
ghz_qc_list = [ghz_state_with_measurement(size, order[:size-1], qubit_depths[:size]) for size in range(1, max_size+1)]

# mqc circuits

In [11]:
def mqc_state_with_measurement(size, order, qubit_depths, phi):
    qc = QuantumCircuit(size, name="mqc_" + str(size) + str(phi))
    qc.compose(ghz_state(size, order, qubit_depths), qubits = range(size), inplace = True)
    qc.barrier()
    qc.x(range(size))
    qc.rz(phi, range(size))
    qc.barrier()
    qc.compose(ghz_state(size, order, qubit_depths).inverse(), qubits = range(size), inplace = True)
    qc.measure_all()
    return qc

In [12]:
mqc_qcs_list = []
for size in range(1, max_size+1):
    mqc_qcs = []
    for j in range(size * 2 + 2):
        phi = (np.pi * j) / (size + 1)
        mqc = mqc_state_with_measurement(size, order[:size-1], qubit_depths[:size], phi)
        mqc_qcs.append(mqc)
    mqc_qcs_list.append(mqc_qcs)

In [13]:
ghz_mqc_mit_qcs_list = []
for i, size in enumerate(range(1, max_size+1)):
    meas_calibs, _ = tensored_meas_cal(mit_pattern=[[j] for j in range(size)], qr=QuantumRegister(size), circlabel='mcal')
    ghz_mqc_mit_qcs = [ghz_qc_list[i]] + mqc_qcs_list[i] + meas_calibs
    ghz_mqc_mit_qcs_list.append(ghz_mqc_mit_qcs)

In [14]:
initial_layouts_list = []
for i, size in enumerate(range(1, max_size+1)):
    initial_layouts = [vertex_list[:size] for _ in range(len(ghz_mqc_mit_qcs_list[i]))]
    initial_layouts_list.append(initial_layouts)

In [15]:
job_list = []
for i, (ghz_mqc_mit_qcs, initial_layouts) in enumerate(zip(ghz_mqc_mit_qcs_list, initial_layouts_list)):
    job = execute(ghz_mqc_mit_qcs,
                  backend=backend, 
                  shots=shots_per_circuit,
                  optimization_level=1,
                  initial_layout=initial_layouts)
    print("the job for", i + 1, "qubit circuits are casted... ( job id:", job.job_id(), ")")
    job_list.append(job.job_id())

the job for 1 qubit circuits are casted... ( job id: 61c8a8b420e2c4e45ceaf74b )
the job for 2 qubit circuits are casted... ( job id: 61c8a8b7a81fcf727dd1d7cc )
the job for 3 qubit circuits are casted... ( job id: 61c8a8b9fa27f609b637cdfa )
the job for 4 qubit circuits are casted... ( job id: 61c8a8ba20e2c4d99aeaf74c )
the job for 5 qubit circuits are casted... ( job id: 61c8a8bce2b9d10d099138e9 )
the job for 6 qubit circuits are casted... ( job id: 61c8a8bdfa27f645ad37cdfb )
the job for 7 qubit circuits are casted... ( job id: 61c8a8bf06d47f3309c41ee5 )
the job for 8 qubit circuits are casted... ( job id: 61c8a8c120e2c4e0f0eaf74d )
the job for 9 qubit circuits are casted... ( job id: 61c8a8c2e2b9d198c59138ea )
the job for 10 qubit circuits are casted... ( job id: 61c8a8c4fa27f6b9df37cdfc )
the job for 11 qubit circuits are casted... ( job id: 61c8a8c5255d4e7631d65de3 )
the job for 12 qubit circuits are casted... ( job id: 61c8a8c73b6df9692073e255 )
the job for 13 qubit circuits are cas

In [16]:
with open("pkls/job_list.pkl", "wb") as f:
    pickle.dump(job_list, f)

In [17]:
with open("pkls/job_list.pkl", "rb") as f:
    print(pickle.load(f))

['61c8a8b420e2c4e45ceaf74b', '61c8a8b7a81fcf727dd1d7cc', '61c8a8b9fa27f609b637cdfa', '61c8a8ba20e2c4d99aeaf74c', '61c8a8bce2b9d10d099138e9', '61c8a8bdfa27f645ad37cdfb', '61c8a8bf06d47f3309c41ee5', '61c8a8c120e2c4e0f0eaf74d', '61c8a8c2e2b9d198c59138ea', '61c8a8c4fa27f6b9df37cdfc', '61c8a8c5255d4e7631d65de3', '61c8a8c73b6df9692073e255', '61c8a8c920e2c4723deaf74e', '61c8a8cb255d4e9801d65de4', '61c8a8cc576354b6bcf51011', '61c8a8cefa27f6aa7537cdfd', '61c8a8d0a81fcf9c5cd1d7ce', '61c8a8d25763547d0ef51012', '61c8a8d4525f9e5b12396d7e', '61c8a8d620e2c4847ceaf74f', '61c8a8da3b6df9950973e256', '61c8a8dd525f9e522b396d7f', '61c8a8e33b6df9ee8273e257', '61c8a8e720e2c4e7b9eaf750', '61c8a8ea06d47fed22c41ee6', '61c8a8ecfa27f63ddc37cdfe', '61c8a8f006d47f8411c41ee7', '61c8a8f33b6df9c36073e258', '61c8a8f606d47f446cc41ee8', '61c8a8faa81fcf5c5ed1d7cf', '61c8a8fee2b9d1c97f9138ec', '61c8a901fa27f6985437cdff', '61c8a906e2b9d1fb839138ed', '61c8a90a255d4e955dd65de7', '61c8a90e20e2c4b884eaf751', '61c8a91306d47fd37b

# device properties

In [5]:
with open("pkls/properties_20211226.pkl", "wb") as f:
    pickle.dump(device.properties(), f)